In [ ]:
import os
import h5py
import napari
import pathlib
import shutil
import tifffile
import scipy.io
import numpy as np
import SessionTools.two_photon as st2p
from matplotlib import pyplot as plt

In [ ]:
# Set experiment details.
superdir = r'C:\Users\ah-pc\Documents\tmp-2p'
#superdir = '/Users/andrewhill/Fisher Lab Dropbox/Andrew Hill/2p-data-processed'
expt = '250818-EPG-8s-5HT'
fly = 'fly2'
session = f'{fly}-005'

# Make paths.
exptdir = pathlib.PurePath(f'{superdir}\{expt}\{fly}\{session}-pv')
point_to_xml_file = pathlib.Path(exptdir.joinpath(f'{session}'))
point_to_tifs = pathlib.Path(exptdir.joinpath(f'{session}-tifs\{session}'))

In [ ]:
#
metadata = st2p.preprocessing.bruker_metadata.read(point_to_xml_file)
metadata['size']['z_planes']=metadata['layout']['frames_per_sequence']

# 
tiff_data = st2p.preprocessing.tiff_tools.read(point_to_tifs, metadata['size'], 
                                               metadata['layout'], first_chan=2)
tiff_data

In [ ]:
h5name = f'{exptdir}/{session}.h5'

if not os.path.exists(h5name):
    st2p.preprocessing.tiff_tools.convert_to_hdf5(tiff_data, h5name, overwrite=True)

f = h5py.File(h5name)
stack = f['/data'][:]
stack.shape

In [ ]:
refimg = st2p.preprocessing.motion_correction.make_ref_img(stack, 0)
refimg.shape

In [ ]:
fig, axs = plt.subplots((refimg.shape[1]),1,figsize=(25,25))

for i in range(refimg.shape[1]):
    axs[i].imshow(refimg[0, i, :, :], cmap='Greys_r', origin='lower'); axs[i].axis('off')

In [ ]:
data_corr, shifts, errors, diffphases = st2p.preprocessing.motion_correction.align_data_chunk(stack,
                                             refimg[0,:,:,:], ref_channel = 0, in_place = True)
data_corr.shape

In [ ]:
maxprojstack = np.max(data_corr, axis=2)
maxprojstack.shape

In [ ]:
nap_stack = data_corr.mean(axis = 1)
np.save(r'C:\Users\ah-pc\py\2p-analysis\napari-stack-txfr\nap-stack.npy', nap_stack)
nap_stack.shape

In [ ]:
# If remote, use local machine to pull nap_stack.npy, do napari things, and push Labels.tif back.

# Do not run next block.

#

In [ ]:
viewer1 = napari.Viewer()
new_layer1 = viewer1.add_image(nap_stack)

# viewer2 = napari.Viewer()
# new_layer2 = viewer2.add_image(maxprojstack.mean(axis = 1))

In [ ]:
shutil.move(r'C:\Users\ah-pc\py\2p-analysis\napari-stack-txfr\Labels.tif', \
            f'{exptdir}\{session}-ROI-mask.tif')

label = tifffile.imread(f"{exptdir}\{session}-ROI-mask.tif")
label = np.squeeze(label)
label.shape

In [ ]:
# If ROI-mask.tif is a single plane that you want projected thru all the layers, use this:

#label = tifffile.imread(f"{exptdir}/{session}-ROI-mask.tif")
#label = np.squeeze(label)
#label = np.repeat(label[np.newaxis, :, :], metadata['size']['z_planes'], axis=0)
#label.shape

# viewerF = napari.Viewer()
# new_layerF1 = viewerF.add_image(np.squeeze(data_corr.mean(axis=1)))
# new_layerF2 = viewerF.add_labels(label)

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(15,15))

axs[0].imshow(maxprojstack.mean(axis=1)[0, :, :], cmap='Greys_r', origin='lower'); axs[0].axis('off')
axs[1].imshow(label.mean(axis=0)[:,:], cmap='Greys_r', origin='lower'); axs[1].axis('off')

In [ ]:
ROI_number = 16
F_array_output = np.zeros((data_corr.shape[1], ROI_number))

for time_point in range(data_corr.shape[1]):
    current_volume = data_corr[0, time_point,:,:]
    for ROI_index in range(1, ROI_number + 1):
        mask = label == ROI_index
        F_array_output[time_point, ROI_index - 1] = current_volume[mask].mean() 

if ROI_number > 1:        
    fig, axs = plt.subplots(ROI_number, 1, figsize=(13, 12))
    for i in range(ROI_number):
        ax = axs[i]
        ax.plot(F_array_output[:,i])
    fig.supylabel('F',fontsize=20)
    plt.xlabel('Frame Number', fontsize=20)
    plt.show()
else:
    plt.figure(figsize= (25,7))
    plt.plot(F_array_output)
    plt.ylabel('F',fontsize=20)
    plt.xlabel('Frame Number', fontsize=20)
    plt.show()

In [ ]:
# Save raw F output as .npy and .mat.
np.save(f'{exptdir}\{session}-F-array.npy', F_array_output)
scipy.io.savemat(f'{exptdir}\{session}-F-array.mat', {'F_array_output': F_array_output})

In [ ]:
# Notes
#
#

In [ ]:
!jupyter nbconvert preprocessing.ipynb --to html

In [ ]:
shutil.move(r'C:\Users\ah-pc\py\2p-analysis\preprocessing.html', f'{exptdir}\preprocessing.html')